## Setup and Optimization Libraries

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# API Configuration
API_BASE = 'http://localhost:3001/api'

print("✓ Parameter Tuning Notebook Initialized")
print(f"API Base: {API_BASE}")

## Section 1: Single Parameter Sensitivity Analysis

Vary one parameter systematically to measure impact

In [ ]:
# Run simulations with varying temperature (affects PCR efficiency)
base_protocol = 'plasmid-prep'
temperature_values = [55, 60, 65, 70, 72, 75]

sensitivity_results = []

print(f"TEMPERATURE SENSITIVITY ANALYSIS")
print(f"Protocol: {base_protocol}")
print("-" * 60)

for temp in temperature_values:
    # Run simulation with specific parameter
    sim_params = {
        'protocol': base_protocol,
        'numRuns': 10,
        'metrics': ['cost', 'duration', 'yield'],
        'parameters': {
            'annealingTemp': temp,
            'numCycles': 35
        }
    }
    
    response = requests.post(
        f'{API_BASE}/v1/agents/simulate',
        json=sim_params
    )
    
    if response.status_code == 200:
        sim_result = response.json()['simulation']
        aggregated = sim_result.get('aggregatedMetrics', {})
        
        result = {
            'temperature': temp,
            'cost_mean': aggregated.get('cost', {}).get('mean', 0),
            'cost_std': aggregated.get('cost', {}).get('std', 0),
            'yield_mean': aggregated.get('yield', {}).get('mean', 0),
            'yield_std': aggregated.get('yield', {}).get('std', 0),
            'duration_mean': aggregated.get('duration', {}).get('mean', 0)
        }
        sensitivity_results.append(result)
        print(f"Temp {temp}°C: Cost ${result['cost_mean']:.2f} | Yield {result['yield_mean']:.1f}% | Duration {result['duration_mean']:.1f}h")

# Create DataFrame
sensitivity_df = pd.DataFrame(sensitivity_results)
print(f"\n✓ Sensitivity analysis complete: {len(sensitivity_df)} temperature points")

## Section 2: Optimization Landscape

Explore cost-yield trade-off space

In [ ]:
# Display optimization landscape
print("TEMPERATURE PARAMETER SWEEP RESULTS:")
print("=" * 80)

display_df = sensitivity_df[['temperature', 'cost_mean', 'yield_mean', 'duration_mean']].copy()
display_df.columns = ['Temp (°C)', 'Cost ($)', 'Yield (%)', 'Duration (h)']
display_df['Efficiency'] = display_df['Yield (%)'] / display_df['Cost ($)']  # Cost-normalized yield

print(display_df.to_string(index=False))

# Find optimal points
print("\n" + "=" * 80)
print("OPTIMIZATION INSIGHTS:")
if not sensitivity_df.empty:
    max_yield_idx = sensitivity_df['yield_mean'].idxmax()
    min_cost_idx = sensitivity_df['cost_mean'].idxmin()
    
    print(f"\nMaximum Yield: {sensitivity_df.loc[max_yield_idx, 'temperature']}°C ({sensitivity_df.loc[max_yield_idx, 'yield_mean']:.1f}%)")
    print(f"Minimum Cost: {sensitivity_df.loc[min_cost_idx, 'temperature']}°C (${sensitivity_df.loc[min_cost_idx, 'cost_mean']:.2f})")
    
    # Cost-efficiency frontier
    efficiency = sensitivity_df['yield_mean'] / sensitivity_df['cost_mean']
    max_eff_idx = efficiency.idxmax()
    print(f"Best Cost-Efficiency: {sensitivity_df.loc[max_eff_idx, 'temperature']}°C (ratio {efficiency[max_eff_idx]:.3f})")

## Section 3: Multi-Parameter Grid Search

Explore combinations of parameters for optimal settings

In [ ]:
# Run grid search over 2 parameters: temperature and cycles
temperatures = [65, 70, 75]
cycles = [30, 35, 40]

grid_results = []

print("MULTI-PARAMETER GRID SEARCH")
print(f"Temperature: {temperatures}")
print(f"PCR Cycles: {cycles}")
print("-" * 80)

for temp in temperatures:
    for cycle in cycles:
        sim_params = {
            'protocol': base_protocol,
            'numRuns': 5,
            'metrics': ['cost', 'duration', 'yield'],
            'parameters': {
                'annealingTemp': temp,
                'numCycles': cycle
            }
        }
        
        response = requests.post(
            f'{API_BASE}/v1/agents/simulate',
            json=sim_params
        )
        
        if response.status_code == 200:
            sim_result = response.json()['simulation']
            aggregated = sim_result.get('aggregatedMetrics', {})
            
            grid_results.append({
                'temperature': temp,
                'cycles': cycle,
                'cost': aggregated.get('cost', {}).get('mean', 0),
                'yield': aggregated.get('yield', {}).get('mean', 0),
                'duration': aggregated.get('duration', {}).get('mean', 0)
            })

grid_df = pd.DataFrame(grid_results)
print(f"\n✓ Grid search complete: {len(grid_df)} parameter combinations")

## Section 4: Parameter Interaction Analysis

Understand how parameters interact and affect outcomes

In [ ]:
# Display grid results as pivot table
print("COST LANDSCAPE ($/reaction):")
print("=" * 60)

if not grid_df.empty:
    cost_pivot = grid_df.pivot_table(values='cost', index='cycles', columns='temperature')
    print(cost_pivot.round(2))
    
    print("\nYIELD LANDSCAPE (%):")
    print("=" * 60)
    yield_pivot = grid_df.pivot_table(values='yield', index='cycles', columns='temperature')
    print(yield_pivot.round(1))
    
    # Correlation analysis
    print("\nPARAMETER CORRELATIONS:")
    print("=" * 60)
    print(f"Temperature-Yield correlation: {grid_df['temperature'].corr(grid_df['yield']):.3f}")
    print(f"Cycles-Yield correlation: {grid_df['cycles'].corr(grid_df['yield']):.3f}")
    print(f"Cycles-Cost correlation: {grid_df['cycles'].corr(grid_df['cost']):.3f}")

## Section 5: Optimization Recommendations

Generate actionable optimization recommendations

In [ ]:
# Generate recommendations
print("OPTIMIZATION RECOMMENDATIONS:")
print("=" * 80)

if not grid_df.empty:
    # Best overall settings
    best_yield_idx = grid_df['yield'].idxmax()
    best_yield = grid_df.loc[best_yield_idx]
    
    print(f"\n1. MAXIMUM YIELD SETTINGS:")
    print(f"   Temperature: {int(best_yield['temperature'])}°C")
    print(f"   PCR Cycles: {int(best_yield['cycles'])}")
    print(f"   Expected Yield: {best_yield['yield']:.1f}%")
    print(f"   Cost: ${best_yield['cost']:.2f}")
    
    # Most cost-efficient
    efficiency = grid_df['yield'] / grid_df['cost']
    best_eff_idx = efficiency.idxmax()
    best_eff = grid_df.loc[best_eff_idx]
    
    print(f"\n2. BEST COST-EFFICIENCY:")
    print(f"   Temperature: {int(best_eff['temperature'])}°C")
    print(f"   PCR Cycles: {int(best_eff['cycles'])}")
    print(f"   Efficiency Ratio: {efficiency[best_eff_idx]:.3f}")
    
    # Minimum cost
    best_cost_idx = grid_df['cost'].idxmin()
    best_cost = grid_df.loc[best_cost_idx]
    
    print(f"\n3. MINIMUM COST SETTINGS:")
    print(f"   Temperature: {int(best_cost['temperature'])}°C")
    print(f"   PCR Cycles: {int(best_cost['cycles'])}")
    print(f"   Cost: ${best_cost['cost']:.2f}")
    print(f"   Expected Yield: {best_cost['yield']:.1f}%")

print("\n" + "=" * 80)
print("Next Steps:")
print("- Validate recommended settings with benchtop experiments")
print("- Run larger scale simulations (100+ runs) to assess variability")
print("- Consider sample-to-sample variation in optimization")

## Key Insights

Parameter optimization enables:
1. **Systematic exploration** of design space
2. **Quantified trade-offs** between competing objectives
3. **Reproducible decision-making** with simulation evidence
4. **Risk assessment** through sensitivity analysis

Use these techniques in your protocol development workflow to accelerate optimization.